In [1]:
import sys, os, re; sys.path = (["../src/", "../"] if re.match(r'^(\w\:\\)|(/)', os.getcwd()) else [])+ sys.path 
from collections import deque

import qubx
%qubxd dev

%load_ext autoreload
%autoreload 2

import tests.qubx.ta.utils_for_testing as test
from qubx.utils import fig, sbp

from qubx.data.readers import (
    CsvDataReader, QuotesDataProcessor, OhlcvDataProcessor, QuotesFromOHLCVDataProcessor, 
    OhlcvPandasDataProcessor, STOCK_DAILY_SESSION
)
from qubx.core.series import TimeSeries, OHLCV
from qubx.ta.indicators import (sma, ema, tema, dema, kama, highest, lowest)

 >  [dev] installing cython rebuilding hook

                   ╻
   ┏┓      ╻       ┃  ┏┓       Quantitative Backtesting Environment 
   ┃┃  ┓┏  ┣┓  ┏┓  ┃  ┏┛       
   ┗┻  ┗┻  ┗┛  ┗   ┃  ┗━       (c) 2024,  ver. Dev
                   ╹       



: 

## <font color='gray'>Reload dev environment</font>

In [7]:
from qubx.core.series import TimeSeries; from qubx.ta.indicators import sma; del sys.modules['qubx.core.series']; del sys.modules['qubx.ta.indicators']
from qubx.core.series import TimeSeries, OHLCV
from qubx.core.utils import recognize_time
from qubx.ta.indicators import (sma, ema, tema, dema, kama, highest, lowest)

## <font color='red'><b>On formed bar experiments</b></font>

In [57]:
# r2x = CsvDataReader('../tests/data/csv/BTCUSDT_ohlcv_M1_sec.csv.gz', QuotesFromOHLCVDataProcessor(trades=0))
# ticks = r2x.read()
# - small storage
r0 = CsvDataReader('../tests/data/csv/quotes.csv', QuotesDataProcessor())
ticks = r0.read()
len(ticks)

500

In [85]:
s0 = TimeSeries('T0', '30Sec', process_every_update=1)
control = TimeSeries('T0', '30Sec')
m0 = ema(s0, 5)
i1 = highest(s0, 5)
for q in ticks:
    s0.update(q.time, 0.5*(q.ask + q.bid))
    control.update(q.time, 0.5*(q.ask + q.bid))

m1 = ema(control, 5)
i11 = highest(control, 5)
mx = test.scols(s0.pd(), control.pd(), m0.pd(), m1.pd(), i1.pd(), i11.pd(), names=[
    'series', 'control', 'streamed', 'finished', 'h1', 'h11'
])#.dropna()
# assert test.N(mx.streamed) == mx.finished
mx

,series,control,streamed,finished,h1,h11
2017-08-24 13:01:00,9.845,9.845,NaN,NaN,NaN,NaN
2017-08-24 13:01:30,10.050,10.050,NaN,NaN,NaN,NaN
2017-08-24 13:02:00,9.950,9.950,NaN,NaN,NaN,NaN
2017-08-24 13:02:30,9.965,9.965,NaN,7.962000,NaN,NaN
2017-08-24 13:03:00,9.785,9.785,9.919000,8.569667,10.050,10.050
2017-08-24 13:03:30,9.685,9.685,9.841000,8.941444,10.050,10.050
2017-08-24 13:04:00,9.675,9.675,9.785667,9.185963,9.965,9.965
2017-08-24 13:04:30,9.815,9.815,9.795444,9.395642,9.965,9.965
2017-08-24 13:05:00,9.865,9.865,9.818630,9.552095,9.865,9.865
2017-08-24 13:05:30,9.915,9.915,9.850753,9.673063,9.915,9.915


In [118]:
s0.pd().ewm(span=5, min_periods=6).mean()

2017-08-24 13:01:00         NaN
2017-08-24 13:01:30         NaN
2017-08-24 13:02:00         NaN
2017-08-24 13:02:30         NaN
2017-08-24 13:03:00         NaN
2017-08-24 13:03:30    9.817571
2017-08-24 13:04:00    9.767093
2017-08-24 13:04:30    9.783711
2017-08-24 13:05:00    9.811531
2017-08-24 13:05:30    9.846629
2017-08-24 13:06:00    9.857883
2017-08-24 13:06:30    9.850196
2017-08-24 13:07:00    9.841754
2017-08-24 13:07:30    9.737479
2017-08-24 13:08:00    9.633081
2017-08-24 13:08:30    9.563615
2017-08-24 13:09:00    9.489001
2017-08-24 13:09:30    9.450975
Name: T0, dtype: float64

In [121]:
s0 = TimeSeries('T0', '30Sec', process_every_update=False)
m0 = ema(s0, 5)
i0 = highest(s0, 5)
for q in ticks: s0.update(q.time, 0.5*(q.ask + q.bid))

In [122]:
s00 = TimeSeries('T0', '30Sec')
for q in ticks: s00.update(q.time, 0.5*(q.ask + q.bid))

In [136]:
np.sin(m0)

array([-0.05440051, -0.12371229, -0.18486457, -0.27519263, -0.35748549, -0.34125346, -0.31196838, -0.24574205, -0.12697301,  0.02913186,  0.23655142,  0.46473342,  0.75464394,  0.99417169,         nan,         nan,         nan])

In [159]:
from qubx.utils.pandas import roll, scols

In [ ]:
scols(s0, m0)

In [157]:
np.nansum((ema(s00, 5) - m0)) == 0

True

In [165]:
c0 = TimeSeries('T0', '30Sec', process_every_update=True)
m1 = ema(c0, 5)
i1 = highest(c0, 5)
for q in ticks:
    c0.update(q.time, 0.5*(q.ask + q.bid))
test.scols(m0.pd(), m1.pd())#.dropna()

,"ema(5,init_mean=True)","ema(5,init_mean=True)"
2017-08-24 13:01:00,NaN,NaN
2017-08-24 13:01:30,NaN,NaN
2017-08-24 13:02:00,NaN,NaN
2017-08-24 13:02:30,7.962000,NaN
2017-08-24 13:03:00,8.569667,9.919000
2017-08-24 13:03:30,8.941444,9.841000
2017-08-24 13:04:00,9.185963,9.785667
2017-08-24 13:04:30,9.395642,9.795444
2017-08-24 13:05:00,9.552095,9.818630
2017-08-24 13:05:30,9.673063,9.850753
